In [14]:
import os
import pandas as pd
SourceDataDict={}
for x in os.listdir("./Data"):
    print(x[:-4])
    SourceDataDict[x[:-4]]=x

BOND_EUR Kraken Historical Data
Crude Oil WTI Futures - Apr 24 (CLJ4)
India 3-Month Bond Yield Historical Data
Namibia 3-Month Bond Yield Historical Data
S&P 500 (US500)
SBI Gold ETF (SBIG)
SPDR® Dow Jones Industrial Average ETF Trust (SPDR)
UK 10 YR Gilt Futures Historical Data
US Soybeans Futures - Mar 24 (ZSH4)
US Wheat Futures - Mar 24 (ZWH4)


In [25]:
import pandas as pd

def load_data(file_path):
    return pd.read_csv(file_path, index_col='Date', parse_dates=True)

import numpy as np

def calculate_returns(df):
    return np.log(df / df.shift(1))

from scipy.optimize import minimize

def portfolio_annual_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights ) *252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns

def min_variance(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def calculate_capm_expected_return(beta, risk_free_rate, market_return):
    return risk_free_rate + beta * (market_return - risk_free_rate)

def plot_efficient_frontier_and_cml(returns, risks, risk_free_rate, market_return, market_risk):
    plt.figure(figsize=(10, 6))
    plt.scatter(risks, returns, c=(returns-risk_free_rate)/risks, marker='o')  # Efficient frontier
    # Capital Market Line
    cml_x = [0, market_risk]
    cml_y = [risk_free_rate, market_return]
    plt.plot(cml_x, cml_y, color='red', label='Capital Market Line', linewidth=2)
    plt.xlabel('Portfolio Risk')
    plt.ylabel('Portfolio Return')
    plt.title('Efficient Frontier and Capital Market Line')
    plt.colorbar(label='Sharpe Ratio')
    plt.show()


def find_tangency_portfolio(portfolios):
    # Assuming 'portfolios' is a DataFrame with columns for 'returns', 'risks', and 'sharpe_ratios'
    max_sharpe_idx = portfolios['sharpe_ratios'].idxmax()
    tangency_portfolio = portfolios.iloc[max_sharpe_idx]
    return tangency_portfolio

def plot_sml(risk_free_rate, market_return, betas, asset_returns):
    plt.figure(figsize=(10, 6))
    market_premium = market_return - risk_free_rate
    sml_y = [risk_free_rate + beta * market_premium for beta in betas]
    plt.plot(betas, sml_y, label='Security Market Line', linestyle='--', color='green')
    plt.scatter(betas, asset_returns, color='blue', label='Assets')
    plt.xlabel('Beta')
    plt.ylabel('Expected Return')
    plt.title('Security Market Line')
    plt.legend()
    plt.show()

def calculate_performance_measures(return_portfolio, std_portfolio, risk_free_rate, beta_portfolio=None):
    sharpe_ratio = (return_portfolio - risk_free_rate) / std_portfolio
    treynor_ratio = (return_portfolio - risk_free_rate) / beta_portfolio if beta_portfolio else None
    return sharpe_ratio, treynor_ratio


In [26]:

# Load Data
# For this example, I'll assume 'data.csv' is your file with closing prices
data = pd.read_csv('./Data/BOND_EUR Kraken Historical Data.csv', index_col='Date', parse_dates=True)
data = data.apply(pd.to_numeric, errors='coerce')
# Example: Fill missing values with the previous value in the column
data.fillna(method='ffill', inplace=True)

# Calculate Returns
returns = calculate_returns(data)

# # Mean-Variance Optimization
mean_returns = returns.mean()
cov_matrix = returns.cov()

# # Assuming a function for optimization (you need to define or adjust it)
# # For example, 'min_variance' could be a function to minimize variance for a given return
optimized_portfolio = min_variance(mean_returns, cov_matrix)

# # CAPM Expected Return Calculation
# # Assuming you have 'beta_values' and other necessary parameters
risk_free_rate = 0.02  # Example risk-free rate
market_return = 0.08  # Example market return

# Plot Efficient Frontier and CML
# Assuming 'plot_efficient_frontier_and_cml' is defined and you have the necessary data
# You would need to generate portfolios to plot, which is not covered in this snippet

# Tangency Portfolio Identification
# Assuming 'find_tangency_portfolio' is defined and you have portfolios data

# Performance Measures Calculation
# Assuming 'calculate_performance_measures' is defined and you have portfolio performance data

# Note: Each of these steps requires specific data and preparatory work not fully covered here.
# For example, generating portfolios for the efficient frontier, calculating betas for CAPM, etc.

# This workflow outlines the process you would follow, with each function being a step in the analysis.
# You'll need to adjust the implementation details based on your data structure and analysis needs.

TypeError: min_variance() missing 1 required positional argument: 'risk_free_rate'